In [1]:
!pip install xlstm==2.0.4 torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [2]:
# ================================
# CELL 1 — Train fusion & Save .pt
# ================================
import os, json, csv, math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch_geometric.data import Batch
from torch_geometric.nn import global_mean_pool, BatchNorm, GCNConv
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, f1_score

# xLSTM imports
from xlstm import (
    xLSTMBlockStack,
    xLSTMBlockStackConfig,
    mLSTMBlockConfig,
    mLSTMLayerConfig,
    sLSTMBlockConfig,
    sLSTMLayerConfig,
    FeedForwardConfig
)

# -----------------------------
# Dataset
# -----------------------------
class MultiModalDataset(Dataset):
    CACHE_FILE = '/kaggle/input/vocab-json/vocab.json'  # nếu có sẵn vocab tại input

    def __init__(self, json_root, pt_root, max_seq_len=1500):
        self.max_len = max_seq_len
        self.samples = []
        if os.path.isfile(self.CACHE_FILE):
            with open(self.CACHE_FILE, 'r') as vf:
                self.vocab = json.load(vf)
            idx = max(self.vocab.values()) + 1
        else:
            self.vocab = {'<PAD>': 0, '<UNK>': 1}
            idx = 2

        mapping = [
            (os.path.join(json_root, 'json-atb-benign-507'), os.path.join(pt_root, 'benign'), 0),
            (os.path.join(json_root, 'ransom-5xx-new', 'ransomware'), os.path.join(pt_root, 'ransomware'), 1)
        ]

        for jdir, pdir, label in mapping:
            if not os.path.isdir(jdir) or not os.path.isdir(pdir):
                continue
            for fname in os.listdir(jdir):
                if not fname.endswith('.json'):
                    continue
                sid = os.path.splitext(fname)[0]
                jpath = os.path.join(jdir, fname)
                ppath = os.path.join(pdir, f"{sid}.pt")
                if not os.path.isfile(ppath):
                    continue
                feat = json.load(open(jpath, 'r'))
                toks = []
                for call in feat.get('api_call_sequence', [])[:1000]:
                    toks.append(f"api:{call.get('api','')}")
                for ft, vals in feat.get('behavior_summary', {}).items():
                    for v in vals:
                        toks.append(f"feature:{ft}:{v}")
                for d in feat.get('dropped_files', []):
                    toks.append(f"dropped_file:{d if not isinstance(d,dict) else d.get('filepath','')}")
                for sig in feat.get('signatures', []):
                    toks.append(f"signature:{sig.get('name','')}")
                for p in feat.get('processes', []):
                    toks.append(f"process:{p.get('name','')}")
                for proto, ents in feat.get('network', {}).items():
                    for e in ents:
                        if isinstance(e, dict):
                            dst = e.get('dst') or e.get('dst_ip', '')
                            port = e.get('dst_port') or e.get('port', '')
                            toks.append(f"network:{proto}:{dst}:{port}")
                        else:
                            toks.append(f"network:{proto}:{e}")
                if not os.path.isfile(self.CACHE_FILE):
                    for t in toks:
                        if t not in self.vocab:
                            self.vocab[t] = idx; idx += 1
                self.samples.append((ppath, toks, label))
        if not os.path.isfile(self.CACHE_FILE):
            json.dump(self.vocab, open(self.CACHE_FILE, 'w'), ensure_ascii=False, indent=2)

    def __len__(self): return len(self.samples)

    def __getitem__(self, i):
        ppath, toks, label = self.samples[i]
        data = torch.load(ppath, weights_only=False)
        idxs = [self.vocab.get(t, self.vocab['<UNK>']) for t in toks]
        if len(idxs) < self.max_len:
            idxs += [self.vocab['<PAD>']] * (self.max_len - len(idxs))
        else:
            idxs = idxs[:self.max_len]
        return data, torch.tensor(idxs), torch.tensor(label, dtype=torch.float32)

def collate_fn(batch):
    gs, seqs, labels = zip(*batch)
    return Batch.from_data_list(gs), torch.stack(seqs), torch.stack(labels)

# -----------------------------
# Models
# -----------------------------
class GCNEncoder(nn.Module):
    def __init__(self, in_feats, hidden=64, drop=0.3):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden)
        self.bn1 = BatchNorm(hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.bn2 = BatchNorm(hidden)
        self.drop = drop
        self.output_dim = hidden

    def forward(self, x, ei, b):
        x = F.relu(self.bn1(self.conv1(x, ei)))
        x = F.dropout(x, self.drop, training=self.training)
        x = F.relu(self.bn2(self.conv2(x, ei)))
        x = F.dropout(x, self.drop, training=self.training)
        return global_mean_pool(x, b)

class xLSTMEncoder(nn.Module):
    def __init__(self, vocab_size, embed=128, seq_len=1500, blocks=1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed, padding_idx=0)
        cfg = xLSTMBlockStackConfig(
            mlstm_block=mLSTMBlockConfig(mlstm=mLSTMLayerConfig(conv1d_kernel_size=4, qkv_proj_blocksize=4, num_heads=4)),
            slstm_block=sLSTMBlockConfig(slstm=sLSTMLayerConfig(backend="vanilla", num_heads=4, conv1d_kernel_size=4,
                                                                bias_init="powerlaw_blockdependent"),
            feedforward=FeedForwardConfig(proj_factor=1.3, act_fn="gelu"),),
            context_length=seq_len, num_blocks=blocks, embedding_dim=embed, slstm_at=[0]
        )
        self.xlstm = xLSTMBlockStack(cfg)
        self.output_dim = embed

    def forward(self, seq):
        return self.xlstm(self.embed(seq)).mean(dim=1)

class MLPClassifier(nn.Module):
    def __init__(self, in_dim, drop=0.3):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, 128), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(128, 64), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(64, 1)
        )
    def forward(self, x): return self.fc(x).squeeze(1)

class MultiModalClassifier(nn.Module):
    def __init__(self, graph_enc, seq_enc, drop=0.3):
        super().__init__()
        self.gcn = graph_enc
        self.xlstm = seq_enc
        fusion_dim = graph_enc.output_dim + seq_enc.output_dim
        self.classifier = MLPClassifier(fusion_dim, drop=drop)
    def forward(self, g, seq, return_emb=False):
        g_emb = self.gcn(g.x, g.edge_index, g.batch)
        s_emb = self.xlstm(seq)
        fused = torch.cat([g_emb, s_emb], dim=1)
        logit = self.classifier(fused)
        if return_emb:
            return logit, g_emb, s_emb, fused
        return logit

# -----------------------------
# Metrics & Train
# -----------------------------
def eval_met(m, l, c, d):
    m.eval(); ls, allp, alll = 0, [], []
    with torch.no_grad():
        for g, seq, labs in l:
            g, seq, labs = g.to(d), seq.to(d), labs.to(d)
            logits = m(g, seq)
            ls += c(logits, labs).item() * labs.size(0)
            p = (torch.sigmoid(logits) > 0.5).float()
            allp.extend([int(x) for x in p.cpu().tolist()])
            alll.extend([int(x) for x in labs.cpu().tolist()])
    cm = confusion_matrix(alll, allp, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    t = tp + tn + fp + fn
    return {'loss': ls/max(t,1), 'acc': (tp+tn)/max(t,1),
            'tpr': tp/(tp+fn+1e-9), 'fpr': fp/(fp+tn+1e-9),
            'f1': f1_score(alll, allp, zero_division=0)}

def run_training_fusion(model, ds, device, tag='GCN+xLSTM'):
    lbls = [lbl for _, _, lbl in ds.samples]
    outer = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
    tv, te = next(outer.split(range(len(ds)), lbls))
    inner = StratifiedShuffleSplit(n_splits=1, test_size=0.17647, random_state=42)
    tr_rel, val_rel = next(inner.split(tv, [lbls[i] for i in tv]))
    tr = [tv[i] for i in tr_rel]; val = [tv[i] for i in val_rel]

    tr_ld = DataLoader(Subset(ds, tr), 8, shuffle=True, collate_fn=collate_fn)
    val_ld = DataLoader(Subset(ds, val), 8, shuffle=False, collate_fn=collate_fn)
    te_ld  = DataLoader(Subset(ds, te), 8, shuffle=False, collate_fn=collate_fn)

    crit = nn.BCEWithLogitsLoss()
    opt  = torch.optim.Adam(model.parameters(), lr=1e-3)

    best_f1, patience, no_imp = 0, 5, 0
    for ep in range(1, 21):
        model.train(); ls, cor, tot = 0, 0, 0
        for g, seq, labs in tr_ld:
            g, seq, labs = g.to(device), seq.to(device), labs.to(device)
            opt.zero_grad()
            logits = model(g, seq)
            loss = crit(logits, labs)
            loss.backward(); opt.step()
            ls += loss.item()*labs.size(0)
            pred = (torch.sigmoid(logits)>0.5).float()
            cor += (pred==labs).sum().item(); tot += labs.size(0)
        trl, tra = ls/max(tot,1), cor/max(tot,1)
        vm = eval_met(model, val_ld, crit, device)
        print(f"[{tag}] Epoch {ep:02d} | Train Loss: {trl:.4f}, Acc: {tra:.4f} | "
              f"Val Loss: {vm['loss']:.4f}, Acc: {vm['acc']:.4f}, F1: {vm['f1']:.4f}")
        if vm['f1'] > best_f1:
            best_f1, o_val = vm['f1'], model.state_dict(); no_imp = 0
        else:
            no_imp += 1
        if no_imp >= patience:
            print(f"[{tag}] Early stopping at epoch {ep}"); break

    model.load_state_dict(o_val)
    tm = eval_met(model, te_ld, crit, device)
    print(f"[{tag}] Test Loss: {tm['loss']:.4f}, Acc: {tm['acc']:.4f}, "
          f"TPR: {tm['tpr']:.4f}, FPR: {tm['fpr']:.4f}, F1: {tm['f1']:.4f}")
    return tr_ld, val_ld, te_ld



In [3]:
# =====================================
# CELL 2 — Inference + XAI (per-sample) — CORRECTED
# =====================================
import os, json, csv, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch
from torch_geometric.nn import global_mean_pool, BatchNorm, GCNConv
from torch_geometric.explain import Explainer, GNNExplainer

In [4]:
# ======= (re)Define Dataset & Models =======
class MultiModalDataset(Dataset):
    def __init__(self, json_root, pt_root, max_seq_len=1500, vocab_json=None):
        self.max_len = max_seq_len
        self.samples = []
        if vocab_json and os.path.isfile(vocab_json):
            with open(vocab_json, 'r') as vf:
                self.vocab = json.load(vf)
        else:
            default_cache = '/kaggle/input/vocab-json/vocab.json'
            if os.path.isfile(default_cache):
                self.vocab = json.load(open(default_cache,'r'))
            else:
                raise FileNotFoundError("No vocab found. Provide vocab_runtime.json or /kaggle/input/vocab-json/vocab.json")

        mapping = [
            (os.path.join(json_root, 'json-atb-benign-507'), os.path.join(pt_root, 'benign'), 0),
            (os.path.join(json_root, 'ransom-5xx-new', 'ransomware'), os.path.join(pt_root, 'ransomware'), 1)
        ]
        for jdir, pdir, label in mapping:
            if not os.path.isdir(jdir) or not os.path.isdir(pdir): continue
            for fname in os.listdir(jdir):
                if not fname.endswith('.json'): continue
                sid = os.path.splitext(fname)[0]
                jpath = os.path.join(jdir, fname)
                ppath = os.path.join(pdir, f"{sid}.pt")
                if not os.path.isfile(ppath): continue
                feat = json.load(open(jpath,'r'))
                toks = []
                for call in feat.get('api_call_sequence', [])[:1000]:
                    toks.append(f"api:{call.get('api','')}")
                for ft, vals in feat.get('behavior_summary', {}).items():
                    for v in vals: toks.append(f"feature:{ft}:{v}")
                for d in feat.get('dropped_files', []):
                    toks.append(f"dropped_file:{d if not isinstance(d,dict) else d.get('filepath','')}")
                for sig in feat.get('signatures', []):
                    toks.append(f"signature:{sig.get('name','')}")
                for p in feat.get('processes', []):
                    toks.append(f"process:{p.get('name','')}")
                for proto, ents in feat.get('network', {}).items():
                    for e in ents:
                        if isinstance(e, dict):
                            dst = e.get('dst') or e.get('dst_ip','')
                            port = e.get('dst_port') or e.get('port','')
                            toks.append(f"network:{proto}:{dst}:{port}")
                        else: toks.append(f"network:{proto}:{e}")
                self.samples.append((ppath, toks, label))

    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        ppath, toks, label = self.samples[i]
        data = torch.load(ppath, weights_only=False)
        idxs = [self.vocab.get(t, self.vocab['<UNK>']) for t in toks]
        if len(idxs) < self.max_len:
            idxs += [self.vocab['<PAD>']] * (self.max_len - len(idxs))
        else:
            idxs = idxs[:self.max_len]
        return data, torch.tensor(idxs), torch.tensor(int(label), dtype=torch.float32)

class GCNEncoder(nn.Module):
    def __init__(self, in_feats, hidden=64, drop=0.3):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden)
        self.bn1 = BatchNorm(hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.bn2 = BatchNorm(hidden)
        self.drop = drop
        self.output_dim = hidden
    def forward(self, x, ei, b):
        x = F.relu(self.bn1(self.conv1(x, ei)))
        x = F.dropout(x, self.drop, training=self.training)
        x = F.relu(self.bn2(self.conv2(x, ei)))
        x = F.dropout(x, self.drop, training=self.training)
        return global_mean_pool(x, b)

class xLSTMEncoder(nn.Module):
    def __init__(self, vocab_size, embed=128, seq_len=1500, blocks=1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed, padding_idx=0)
        cfg = xLSTMBlockStackConfig(
        mlstm_block=mLSTMBlockConfig(
            mlstm=mLSTMLayerConfig(conv1d_kernel_size=4, qkv_proj_blocksize=4, num_heads=4)
        ),
        slstm_block=sLSTMBlockConfig(
            slstm=sLSTMLayerConfig(
                backend="vanilla",
                num_heads=4,
                conv1d_kernel_size=4,
                bias_init="powerlaw_blockdependent",
            ),
            feedforward=FeedForwardConfig(proj_factor=1.3, act_fn="gelu"), 
        ),
        context_length=seq_len,
        num_blocks=blocks,
        embedding_dim=embed,
        slstm_at=[0],
    )

        from xlstm import xLSTMBlockStack
        self.xlstm = xLSTMBlockStack(cfg)
        self.output_dim = embed
    def forward(self, seq):
        return self.xlstm(self.embed(seq)).mean(dim=1)

class MLPClassifier(nn.Module):
    def __init__(self, in_dim, drop=0.3):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, 128), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(128, 64), nn.ReLU(), nn.Dropout(drop),
            nn.Linear(64, 1)
        )
    def forward(self, x): return self.fc(x).squeeze(1)

class MultiModalClassifier(nn.Module):
    def __init__(self, graph_enc, seq_enc, drop=0.3):
        super().__init__()
        self.gcn = graph_enc
        self.xlstm = seq_enc
        fusion_dim = graph_enc.output_dim + seq_enc.output_dim
        self.classifier = MLPClassifier(fusion_dim, drop=drop)
    def forward(self, g, seq, return_emb=False):
        g_emb = self.gcn(g.x, g.edge_index, g.batch)
        s_emb = self.xlstm(seq)
        fused = torch.cat([g_emb, s_emb], dim=1)
        logit = self.classifier(fused)
        if return_emb:
            return logit, g_emb, s_emb, fused
        return logit

In [5]:
# ---------- helpers ----------
def get_index_by_sid(ds, sid: str):
    target = f"{sid}.pt"
    for i, (ppath, _, _) in enumerate(ds.samples):
        if ppath.endswith(target):
            return i
    return None

def make_single_batch(ds, index: int, device):
    data, seq, label = ds[index]
    g = Batch.from_data_list([data]).to(device)
    seq = seq.unsqueeze(0).to(device)
    return g, seq, int(label.item())

@torch.no_grad()
def _get_embeddings(model, g, seq):
    model.eval()
    logits, g_emb, s_emb, fused = model(g, seq, return_emb=True)
    return logits, g_emb.detach(), s_emb.detach(), fused.detach()

def integrated_gradients_fused(model, g, seq, device, steps=64, baseline='zero'):
    model.eval()
    _, g_emb, s_emb, fused = _get_embeddings(model, g, seq)
    if baseline == 'zero':
        base = torch.zeros_like(fused)
    elif baseline == 'mean':
        base = fused.mean(dim=0, keepdim=True).repeat(fused.size(0), 1)
    else:
        base = torch.zeros_like(fused)
    total_attr = torch.zeros_like(fused)
    for alpha in torch.linspace(0, 1, steps, device=device):
        x = base + alpha * (fused - base)
        x.requires_grad_(True)
        logits = model.classifier(x)
        grads = torch.autograd.grad(logits.sum(), x, retain_graph=False)[0]
        total_attr += grads
    ig = (fused - base) * (total_attr / steps)
    return ig, fused, g_emb, s_emb

def branch_scores_from_ig(ig, g_dim):
    g_attr = ig[:, :g_dim]
    s_attr = ig[:, g_dim:]
    g_score = g_attr.abs().sum(dim=1)
    s_score = s_attr.abs().sum(dim=1)
    return g_score, s_score



def explain_graph_simple(model, data_single, device, topk=5):
    """Simplified graph explanation using gradient-based importance"""
    data_single = data_single.to(device)
    data_single.x.requires_grad_(True)
    
    # Forward pass
    graph_emb = model.gcn(data_single.x, data_single.edge_index, data_single.batch)
    
    # Use only graph part of the classifier for attribution
    with torch.no_grad():
        fc0 = model.classifier.fc[0]
        W_left = fc0.weight[:, :model.gcn.output_dim].mean(dim=0, keepdim=True).T
    
    score = (graph_emb @ W_left).sum()
    
    # Compute gradients
    score.backward()
    
    # Node importance based on gradient magnitude
    node_imp = data_single.x.grad.abs().sum(dim=1)
    
    # For edge importance, we can use a simple heuristic based on connected nodes
    edge_imp = torch.zeros(data_single.edge_index.size(1), device=device)
    for i, (src, dst) in enumerate(data_single.edge_index.t()):
        edge_imp[i] = (node_imp[src] + node_imp[dst]) / 2
    
    node_imp = node_imp.detach().cpu()
    edge_imp = edge_imp.detach().cpu()
    
    k_nodes = min(topk, node_imp.numel())
    k_edges = min(topk, edge_imp.numel())
    top_nodes = node_imp.topk(k_nodes).indices.tolist()
    top_edges = edge_imp.topk(k_edges).indices.tolist()
    
    return top_nodes, top_edges, node_imp, edge_imp

def explain_sequence_single(model, seq_batch, device, ds, topk=20, steps=64):
    emb_layer = model.xlstm.embed
    with torch.no_grad():
        emb = emb_layer(seq_batch).detach()
    base = torch.zeros_like(emb)
    total_attr = torch.zeros_like(emb)
    for alpha in torch.linspace(0, 1, steps, device=device):
        x = base + alpha * (emb - base)
        x.requires_grad_(True)
        s_out = model.xlstm.xlstm(x)
        s_pooled = s_out.mean(dim=1)
        g_zero = torch.zeros(s_pooled.size(0), model.gcn.output_dim, device=device)
        fused_ = torch.cat([g_zero, s_pooled], dim=1)
        logit_ = model.classifier(fused_)
        grads = torch.autograd.grad(logit_.sum(), x, retain_graph=False)[0]
        total_attr += grads
    ig = (emb - base) * (total_attr / steps)
    token_scores = ig.abs().sum(dim=2).squeeze(0)  # [T]
    
    # Move everything to CPU for indexing
    token_scores = token_scores.cpu()
    pad_id = ds.vocab['<PAD>']
    id2tok = {v: k for k, v in ds.vocab.items()}
    tokens = seq_batch.squeeze(0).detach().cpu()
    valid = (tokens != pad_id).nonzero(as_tuple=False).view(-1)
    
    if valid.numel() == 0:
        return []
    
    k = min(topk, valid.numel())
    top = token_scores[valid].topk(k)
    sel_idx = valid[top.indices.cpu()].tolist()  # Ensure indices are on CPU
    
    return [(id2tok.get(int(tokens[i]), str(int(tokens[i]))), float(token_scores[i].item())) for i in sel_idx]

def analyze_graph_structure(data_single, top_nodes, top_edges, save_path='/kaggle/working/graph_analysis.csv'):
    """Phân tích chi tiết cấu trúc đồ thị cho các node và edge quan trọng"""
    analysis = []
    
    # Phân tích top nodes
    print("=== TOP NODES ANALYSIS ===")
    for i, node_idx in enumerate(top_nodes, 1):
        if node_idx < data_single.x.size(0):
            node_features = data_single.x[node_idx]
            analysis.append({
                'type': 'node',
                'rank': i,
                'node_id': node_idx,
                'feature_summary': f"Feature dim: {node_features.size(0)}, Mean: {node_features.mean().item():.3f}",
                'degree': (data_single.edge_index[0] == node_idx).sum().item()  # Số kết nối
            })
            print(f"Node {node_idx}: Degree={analysis[-1]['degree']}, Features: {analysis[-1]['feature_summary']}")
    
    # Phân tích top edges
    print("\n=== TOP EDGES ANALYSIS ===")
    for i, edge_idx in enumerate(top_edges, 1):
        if edge_idx < data_single.edge_index.size(1):
            src_node = data_single.edge_index[0, edge_idx].item()
            dst_node = data_single.edge_index[1, edge_idx].item()
            analysis.append({
                'type': 'edge', 
                'rank': i,
                'edge_id': edge_idx,
                'source_node': src_node,
                'target_node': dst_node,
                'connection': f"{src_node} -> {dst_node}"
            })
            print(f"Edge {edge_idx}: {src_node} → {dst_node}")
    
    # Lưu kết quả
    import csv
    with open(save_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['type', 'rank', 'node_id', 'feature_summary', 'degree', 'edge_id', 'source_node', 'target_node', 'connection'])
        writer.writeheader()
        writer.writerows(analysis)
    
    return analysis

def interpret_graph_patterns(analysis, top_nodes, top_edges):
    """Diễn giải các mẫu đồ thị quan trọng"""
    print("\n=== GRAPH PATTERN INTERPRETATION ===")
    
    # Tìm các node có nhiều kết nối (hub nodes)
    hub_nodes = [item for item in analysis if item['type'] == 'node' and item['degree'] > 10]
    if hub_nodes:
        print(f"🔗 HUB NODES (high connectivity): {[n['node_id'] for n in hub_nodes]}")
        print("   → Các node này có thể là processes chính hoặc critical system resources")
    
    # Phân tích các kết nối quan trọng
    critical_edges = []
    for edge_info in [item for item in analysis if item['type'] == 'edge']:
        src, dst = edge_info['source_node'], edge_info['target_node']
        # Kiểm tra nếu kết nối giữa các top nodes
        if src in top_nodes and dst in top_nodes:
            critical_edges.append((src, dst))
    
    if critical_edges:
        print(f"🔴 CRITICAL CONNECTIONS between top nodes: {critical_edges}")
        print("   → Đây là các kết nối quan trọng nhất trong đồ thị")
    
    # Tìm star patterns (một node kết nối với nhiều node khác)
    from collections import defaultdict
    connection_count = defaultdict(int)
    for edge_info in [item for item in analysis if item['type'] == 'edge']:
        connection_count[edge_info['source_node']] += 1
    
    star_nodes = [node for node, count in connection_count.items() if count >= 3]
    if star_nodes:
        print(f"⭐ STAR PATTERN NODES: {star_nodes}")
        print("   → Các node này kết nối với nhiều node khác, có thể là central processes")

def predict_and_explain_one(fusion, ds, device, sid=None, index=None,
                            save_prefix='/kaggle/working/single_sample'):
    assert (sid is not None) or (index is not None), "Cần truyền sid hoặc index"
    if sid is not None:
        index_ = None
        target = f"{sid}.pt"
        for i, (ppath, _, _) in enumerate(ds.samples):
            if ppath.endswith(target): index_ = i; break
        assert index_ is not None, f"Không tìm thấy sid='{sid}'"
        index = index_
    g, seq, label = make_single_batch(ds, index, device)
    fusion.eval()
    with torch.no_grad():
        logit = fusion(g, seq)
        prob = torch.sigmoid(logit).item()
        pred = int(prob >= 0.5)
    print(f"[Inference] index={index}, label={label}, prob={prob:.4f}, pred={pred}")

    ig, fused, g_emb, s_emb = integrated_gradients_fused(fusion, g, seq, device, steps=64, baseline='zero')
    g_dim = fusion.gcn.output_dim
    g_score, s_score = branch_scores_from_ig(ig, g_dim)
    g_s, s_s = float(g_score.item()), float(s_score.item())
    dominant = 'graph' if g_s >= s_s else 'seq'
    print(f"[XAI-L1] graph_score={g_s:.3f} | seq_score={s_s:.3f} | dominant={dominant}")

    with open(f"{save_prefix}_branch.csv", 'w', newline='', encoding='utf-8') as f:
        w = csv.DictWriter(f, fieldnames=['index','label','prob','pred','graph_score','seq_score','dominant'])
        w.writeheader()
        w.writerow({'index': index, 'label': label, 'prob': prob, 'pred': pred,
                    'graph_score': g_s, 'seq_score': s_s, 'dominant': dominant})
    print(f"[SAVE] {save_prefix}_branch.csv")

    data_i = g.to_data_list()[0]
    data_i.batch = torch.zeros(data_i.x.size(0), dtype=torch.long, device=device)
    
    # Use simplified graph explanation
    try:
        top_nodes, top_edges, node_imp, edge_imp = explain_graph_simple(fusion, data_i, device, topk=5)
        print(f"[XAI-L2 Graph] top_nodes={top_nodes} | top_edges_idx={top_edges}")
        
        # 🔥 THÊM PHÂN TÍCH CHI TIẾT ĐỒ THỊ
        graph_analysis = analyze_graph_structure(data_i, top_nodes, top_edges, 
                                               save_path=f"{save_prefix}_graph_analysis.csv")
        interpret_graph_patterns(graph_analysis, top_nodes, top_edges)
        
    except Exception as e:
        print(f"[XAI-L2 Graph] Failed: {e}")
        top_nodes, top_edges = [], []


    top_tokens = explain_sequence_single(fusion, seq, device, ds, topk=20, steps=64)
    preview = ', '.join([f"{t}:{s:.3f}" for t, s in top_tokens[:10]])
    print(f"[XAI-L2 Seq] top_tokens (preview 10): {preview}")
    with open(f"{save_prefix}_sequence_tokens.csv", 'w', newline='', encoding='utf-8') as f:
        w = csv.DictWriter(f, fieldnames=['rank','token','score'])
        w.writeheader()
        for r, (t, s) in enumerate(top_tokens, 1):
            w.writerow({'rank': r, 'token': t, 'score': s})
    print(f"[SAVE] {save_prefix}_sequence_tokens.csv")

    return {'index': index, 'label': label, 'prob': prob, 'pred': pred,
            'branch': {'graph_score': g_s, 'seq_score': s_s, 'dominant': dominant},
            'graph_top_nodes': top_nodes, 'graph_top_edges_idx': top_edges,
            'seq_top_tokens': top_tokens}

# ===== Load model & dataset, then run =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load dataset (dùng vocab_runtime.json đã save ở Cell 1)
jr, pr = '/kaggle/input', '/kaggle/input/1000-final/1000'
ds = MultiModalDataset(jr, pr, vocab_json="/kaggle/working/vocab_runtime.json")

# Build model
g_feats = ds[0][0].x.size(1)
fusion = MultiModalClassifier(GCNEncoder(g_feats).to(device),
                              xLSTMEncoder(len(ds.vocab)).to(device)).to(device)

# Load weights
fusion.load_state_dict(torch.load("/kaggle/input/fusion-pt/fusion_model.pt", map_location=device))
fusion.eval()
print("Loaded /kaggle/input/fusion-pt/fusion_model.pt")

# Inference + XAI cho 1 mẫu (đổi index hoặc dùng sid='ten_file')
result = predict_and_explain_one(fusion, ds, device, index=2)
result

Loaded /kaggle/input/fusion-pt/fusion_model.pt
[Inference] index=2, label=0, prob=0.5348, pred=1


/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[XAI-L1] graph_score=14.459 | seq_score=14.824 | dominant=seq
[SAVE] /kaggle/working/single_sample_branch.csv
[XAI-L2 Graph] top_nodes=[1570, 1540, 1566, 1569, 1541] | top_edges_idx=[2383, 2379, 2381, 2450, 2449]
=== TOP NODES ANALYSIS ===
Node 1570: Degree=68, Features: Feature dim: 768, Mean: -0.008
Node 1540: Degree=857, Features: Feature dim: 768, Mean: -0.009
Node 1566: Degree=2, Features: Feature dim: 768, Mean: -0.008
Node 1569: Degree=2, Features: Feature dim: 768, Mean: -0.008
Node 1541: Degree=144, Features: Feature dim: 768, Mean: -0.009

=== TOP EDGES ANALYSIS ===
Edge 2383: 1570 → 1540
Edge 2379: 1566 → 1540
Edge 2381: 1569 → 1540
Edge 2450: 1570 → 1541
Edge 2449: 1570 → 1340

=== GRAPH PATTERN INTERPRETATION ===
🔗 HUB NODES (high connectivity): [1570, 1540, 1541]
   → Các node này có thể là processes chính hoặc critical system resources
🔴 CRITICAL CONNECTIONS between top nodes: [(1570, 1540), (1566, 1540), (1569, 1540), (1570, 1541)]
   → Đây là các kết nối quan trọng nhấ

{'index': 2,
 'label': 0,
 'prob': 0.5348159670829773,
 'pred': 1,
 'branch': {'graph_score': 14.458660125732422,
  'seq_score': 14.823923110961914,
  'dominant': 'seq'},
 'graph_top_nodes': [1570, 1540, 1566, 1569, 1541],
 'graph_top_edges_idx': [2383, 2379, 2381, 2450, 2449],
 'seq_top_tokens': [('api:GetSystemTimeAsFileTime', 0.4028666019439697),
  ('api:NtOpenKey', 0.372043251991272),
  ('feature:regkey_read:HKEY_LOCAL_MACHINE\\SOFTWARE\\Microsoft\\Fusion\\NativeImagesIndex\\v2.0.50727_64\\IL\\7950e2c5\\19b8f67f\\82\\DisplayName',
   0.3290649652481079),
  ('api:GetSystemTimeAsFileTime', 0.32440558075904846),
  ('feature:file_opened:C:\\', 0.3222436308860779),
  ('feature:file_failed:C:\\Windows\\Microsoft.NET\\Framework64\\Upgrades.2.0.50727\\',
   0.30211156606674194),
  ('feature:regkey_read:HKEY_LOCAL_MACHINE\\SOFTWARE\\Microsoft\\Windows NT\\CurrentVersion\\MiniDumpAuxiliaryDlls\\C:\\Windows\\system32\\RPCRT4.dll',
   0.30157166719436646),
  ('feature:file_opened:C:\\Windows\\